# Halloween Viz EDA
In this notebook, I am going to explore the [Halloween Viz dataset](https://www.dataplusscience.com/HalloweenData.html) created by [Jeffrey A. Shaffer](https://twitter.com/HighVizAbility), a resident of Cincinnati on the number of candies given out on Halloween each year. The dataset is further broken down into number of candies given out each half an hour between the trick-or-treat timings (6 to 8 p.m.). Jeffrey is an _"Author, Data Viz Professor, Tableau Zen Master, Data Mining Geek, Recovering Musician"_ (Source: Twitter).

## Creating a Story
This dataset is simple and can can be combined with other external datasets to provide better insight into the trick-or-treat figures. I will be considering [IMDB's horror movies dataset](https://www.kaggle.com/PromptCloudHQ/imdb-horror-movie-dataset) to build on the halloween dataset. I will be counting the number of movies released in a paritcular year and the average IMDB rating of the movie.
> **The relationship between the number of _good_ horror movies released and number of candies distributed in a particular can be explored.**

Later, a simple regression model can be used to make predicitons on what the demand for candies will be like in 2021.

_Note: As 2020 had been the year of the pandemic, there should be a sharp rise in the number of trick-or-treaters in 2021._


# Loading Libraries

In [20]:
import pandas as pd
from pandas_profiling import ProfileReport
import pathlib
import os
import matplotlib.pyplot as plt

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Loading Data

In [9]:
dir = "/content"
filename = "IMDB Horror movies.csv"
path = pathlib.Path(os.path.join(dir, filename))

if not path.exists():
    raise FileNotFoundError(f"No file named {filename} is present at the location provided. Path: {path}")

if path.suffix == ".xlsx":
    horror_df = pd.read_excel(path)
elif path.suffix == ".csv":
    horror_df = pd.read_csv(path)

horror_df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",NaN
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,NaN,"$30,000"
2,Sleepwalking (2017),Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...",Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",NaN
3,Treasure Chest of Horrors II (2013),Comedy| Horror| Thriller,23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...",Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",NaN
4,Infidus (2015),Crime| Drama| Horror,10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...",Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,NaN,NaN


In [21]:
profile = ProfileReport(halloween_df)

TypeError: ignored

In [11]:
horror_df.profile_report()

AttributeError: ignored